In [1]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import warnings
import lightgbm as lgb
from workflow import main_indicators,Larry_simples_inverso,Larry_simples_compra,Larry_completo
from sklearn.model_selection import train_test_split
import plotly.express as px
from datetime import datetime
warnings.filterwarnings("ignore")

In [12]:
ativo = 'win_15_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)

#df = df.iloc[:1000]

In [2]:
ativo = 'EURUSD1'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', sep = '\t', names = colnames)

In [13]:
df = main_indicators(df)

df_acao = Larry_completo(df, tempo_fechar=-5, excluir_seguidos = True, acao = 'all', x_stop = 2)

#df_acao = df_acao[df_acao['tend'] == 0]

compra = df_acao[df_acao['acao'] =='call']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='call'].shape[0]
venda = df_acao[df_acao['acao'] =='sell']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='sell'].shape[0]

compra_result = df_acao[df_acao['acao'] =='call']['resultado_valor'].sum()
venda_result = df_acao[df_acao['acao'] =='sell']['resultado_valor'].sum()

print(f'Venda: {venda}% -- Compra: {compra}%')
print(f'Venda: {venda_result} -- Compra: {compra_result}')

Venda: 0.5069486404833837% -- Compra: 0.530705079605762%
Venda: 35800.0 -- Compra: 78830.0


In [15]:
df_acao = df_acao.drop(columns = ['MA_9_diff_value', 'MA_9_tend', 'MA_9_tend_diff', 'preco','stop', 'realizacao_simplista', 'stop_real', 'tend'])

In [22]:
df_train = df_acao.set_index(['Data','resultado_valor'])

y = df_train['resultado_binario']
train = df_train.drop(columns = ['resultado_binario','acao'])

In [30]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [27]:
x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.3)

In [31]:
clf = lgb.LGBMClassifier()
clf.fit(x_train, y_train)

# predict the results
y_pred=clf.predict(x_test)

# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

print('Training set score: {:.4f}'.format(clf.score(x_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(x_test, y_test)))

LightGBM Model accuracy score: 0.4647
Training set score: 0.9933
Test set score: 0.4647


In [33]:
df_train['resultado_binario'].sum()/df_train.shape[0]

0.5174848688634835